In [1]:
from flask import Flask, request, render_template
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import os

In [2]:
#Creat an app object using the flask class
app = Flask(__name__)

In [3]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
portStemmer = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\OMEN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OMEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Load model and vectorizer
model_path = os.path.join('models', 'model.pkl')
vectorizer_path = os.path.join('models', 'vectorizer.pkl')

try:
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    with open(vectorizer_path, 'rb') as file:
        vectorizer = pickle.load(file)
except Exception as e:
    model = None
    vectorizer = None
    print(f"Error loading model/vectorizer: {e}")

In [5]:
def clean(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

def preprocess(text):
    text = clean(text)
    tokens = [portStemmer.stem(word) for word in text.split() if word not in stop_words]
    return ' '.join(tokens)

In [6]:
@app.route('/')
def home():
    return render_template('index.html')

In [7]:
#Define the route to be home 
#The decorator below links the relative route of the URL to the function it is dec
#Here, home function is with '/', our root directory.
# Running the app sends us to index.html.
# Note that the render_template means it looks for the file in the templates folder. 
#use the route() decorator to tell Flask what URL should trigger our function.
@app.route('/predict', methods=['POST'])
def predict():
    if model is None or vectorizer is None:
        return render_template('index.html', prediction_text='Model not loaded correctly.')

    try:
        article_text = request.form['article']
        processed_text = preprocess(article_text)
        features = vectorizer.transform([processed_text])
        prediction = model.predict(features)

        output = "REAL" if prediction[0] == 1 else "FAKE"
        return render_template('index.html', prediction_text=f'This news is {output}')
    except Exception as e:
        print(f"Error during prediction: {e}")
        return render_template('index.html', prediction_text='Error during prediction.')


if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Apr/2025 14:11:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2025 14:11:39] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Apr/2025 14:11:43] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2025 14:11:57] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2025 14:11:59] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2025 14:12:02] "POST /predict HTTP/1.1" 200 -
